In [1]:
from sage.matroids.advanced import *

In [2]:
# r, an integer vector of length n where r_i is the number of states of the ith random variable in the model
# delta, a simplicial complex given by its facets
# Returns the matrix naturally associated to the toric ideal of the Hierarchical model defined by r and delta
# The columns of this matrix are the vertices of the marginal polytope
# must index vertices from 0
def hierMatrix(r,delta):
    
    # The final matrix that will be output
    modelMapMatrix = []
    
    # Loop over all possible states at the vertices of the complex
    for complexState in cartesian_product([range(statesAtVertex) for statesAtVertex in r]):
        
        # The row of the matrix corresponding to vertexStates
        curRow = []
         
        # Build the row one facet at a time looping over each facet
        for facet in delta:
            
            # The part of curRow corresponding to this facet
            facetRow = []
            
            # Loop over the possible states the facet could take
            for facetState in cartesian_product([range(r[i]) for i in facet]):
                
                # Sets a coordinate to 1 if the facetState corresponds to the complexState
                if ( list(facetState) == [complexState[i] for i in facet] ):
                    
                    facetRow += [1]
                    
                else:
                    
                    facetRow += [0]
                    
            # Adds the part of the row corresponding to this facet to the total row
            curRow += facetRow
            
        # Adds the row corresponding to complexState to the matrix
        modelMapMatrix.append(curRow)
        
    return matrix(modelMapMatrix).transpose()

In [3]:
# G, a graph object of the form ([n],E)
# returns the list of cuts of the graph G = ([n],E)
def getCuts(G):
    
    cuts = []
    
    for setPartition in SetPartitions(len(G.vertices()),2):
        
        A = setPartition[0]
        B = setPartition[1]
        
        cutEdges = []
        
        for edge in G.edge_iterator():
            
            if ( (edge[0] in A and edge[1] in B) or (edge[0] in B and edge[1] in A) ):
                
                cutEdges.append(edge)
                
        cuts.append(cutEdges)
        
    return cuts

In [4]:
# G, a graph object of the form ([n],E)
# returns the indicator vectors for each cut as columns of the matrix
# the columns are the vertices of the cut polytope of G
def cutMatrix(G):
    
    cuts = getCuts(G)
    
    cutVectors = []
    
    cutVectors.append([0 for i in range(len(G.edges()))])
    
    for cut in cuts:
        
        cutVector = []
        
        for edge in G.edge_iterator():
            
            if ( edge in cut ):
                
                cutVector.append(1)
                
            else: 
                
                cutVector.append(0)
                
        
        cutVectors.append(cutVector)
        
    return matrix(cutVectors).transpose()

In [5]:
# delta, a simplicial complex given by its facets
# Returns a matrix whose columns are the vertices of the correlation polytope of delta
def corrMatrix(delta):
    
    verts = []
    
    simpComp = SimplicialComplex(delta)
    
    for S in Subsets(simpComp.vertices()):
        
        vertS = []
        
        for face in gradedLexFaces(delta)[1:]:

            faceCoordinate = 0

            if Set(face).issubset(Set(S)):

                faceCoordinate = 1

            vertS.append(faceCoordinate)
            
        
        verts.append(vertS)
        
    
    return matrix(verts).transpose()

In [6]:
# delta, a simplicial complex given by its facets
# Returns a matrix whose columns are the vertices of the generalized cut polytope of delta
def genCutMatrix(delta):
    
    verts = []
    
    simpComp = SimplicialComplex(delta)
    
    for S in Subsets(simpComp.vertices()):
        
        vertS = []
        
        for face in gradedLexFaces(delta)[1:]:
                
            faceCoordinate = 0
                
            if (not 2.divides(Set(face).intersection(Set(S)).cardinality())):
                    
                faceCoordinate = 1
                    
            vertS.append(faceCoordinate)
                
                
        verts.append(vertS)
        
    
    return matrix(verts).transpose()


In [7]:
# delta, a simplicial complex on [n] given by its facets
# S, a subset of [n]
# creates the vertex d^S of Gcut(delta)
def genCutVert(delta, S):
    
    vert = []
    
    for face in gradedLexFaces(delta)[1:]:
                
        faceCoordinate = 0
                
        if (not 2.divides(Set(face).intersection(Set(S)).cardinality())):
                    
            faceCoordinate = 1
                    
        vert.append(faceCoordinate)
        
    return vert

In [8]:
# delta, a simplicial complex given by its facets
# Returns the matrix representing the generalized covariance map that sends Corr(delta) to Cut(delta)
def genCovMap(delta):
    
    rows = []
    
    simpComp = SimplicialComplex(delta)
    
    
    for rowDim in range(0, simpComp.dimension() + 1):
            
        for rowFace in sorted(list(simpComp.faces()[rowDim])):
            
            curRow = []
            
            for columnDim in range(0, simpComp.dimension() + 1):
                
                for columnFace in sorted(list(simpComp.faces()[columnDim])):
                    
                    curEntry = 0
                    
                    if (Set(columnFace).issubset(Set(rowFace))):
                        
                        curEntry = (-2)^(Set(columnFace).cardinality() - 1)
                    
                    curRow.append(curEntry)
                    
                    
            rows.append(curRow)
    
    return matrix(rows) 

In [9]:
# These functions just use the above matrices to construct a polytope
# Nothing special is going on here they are just meant to be convienent
# The transpose is there because the function Polyhedron(vertices = matrix) uses the rows as vertices not the columns
# taking the transpose gives us a polytope with the columns of the above matrices as the vertices which is what we want

def marginalPolytope(r, delta):
    
    return Polyhedron(vertices = hierMatrix(r, delta).transpose())

def cutPolytope(G):
    
    return Polyhedron(vertices = cutMatrix(G).transpose())

def corrPolytope(delta):
    
    return Polyhedron(vertices = corrMatrix(delta).transpose())

def genCutPolytope(delta):
    
    return Polyhedron(vertices = genCutMatrix(delta).transpose())



In [10]:
# delta, a simplicial complex given by its facets
# dimension, default value is -1. If set to some other value it will only return the faces of that dimension
# Returns a list of all of the faces of delta in graded lex order
def gradedLexFaces(delta, dimension = -1):
    
    faces = []
    
    simpComp = SimplicialComplex(delta)
    
    if (dimension != -1):
        
        for face in sorted(list(simpComp.faces()[dimension])):
            
            faces.append(face)
            
    else:
    
        for dim in range(-1, simpComp.dimension() + 1):

            for face in sorted(list(simpComp.faces()[dim])):

                faces.append(face)
            
    return faces

In [13]:
# delta, a simplicial complex on [n] given as a list of facets
# returns a list of the support of each facet of Gcut(delta), 
# that is for each facet defining inequality of Gcut(delta) it gives a list of faces of delta
# whose corresponding entry in the inequality is nonzero
def facesInHrep(delta):
    
    facesInA = []
    
    hRep = genCutPolytope(delta).Hrep_generator()
    
    for relation in hRep:
        
        a = relation.A()
        
        facesInRelation = []
        
        for i in range(len(a)):
            
            if (a[i] != 0):
                
                facesInRelation.append(gradedLexFaces(delta)[i+1])
        
        facesInA.append(facesInRelation)
        
    return facesInA

In [14]:
# delta, a simplicial complex on [n] given as a list of facets
# Same as facesinHrep but only returns a list for each facet up to switching
def facesInHrepModSwitch(delta):
    
    facesInA = []
    
    hRep = [a[0] for a in ineqsModSwitch(delta, True, False)[0]]
    
    for a in hRep:
        
        facesInRelation = []
        
        for i in range(len(a)):
            
            if (a[i] != 0):
                
                facesInRelation.append(gradedLexFaces(delta)[i+1])
        
        facesInA.append(facesInRelation)
        
    return facesInA

In [15]:
# P, a polytope
# l, a linear functional given as a list
# c, a constant
# checks that lx \leq c is a valid inequality for the polytope P
def isValidIneq(P, l, c):
    
    for vertex in P.vertices():
        
        if ( vector(l).dot_product(vector(vertex)) > c):
            
            return false

        
    return True

In [18]:
# delta, a simplicial complex given by its facets
# Displays the inequalities of Gcut(delta) nicely in the d_F coordinates
def ineqsToString(delta):
    
    hRep = genCutPolytope(delta).Hrep_generator()
    
    ineqs = []
    
    for relation in hRep:
        
        a = relation.A()
        b = relation.b()
        
        i = 0
        relationStr = ''
        
        for face in gradedLexFaces(delta)[1:]:
            
            if (a[i] > 0 and a[i] == 1):
                
                relationStr += '-d' + ''.join(str(j) for j in face) + ' + '
                
            elif (a[i] > 0 and a[i] != 1):
                    
                relationStr += str(-a[i]) + 'd' + ''.join(str(j) for j in face) + ' + '
                    
                    
            elif (a[i] < 0 and a[i] == -1):
                
                relationStr += 'd' + ''.join(str(j) for j in face) + ' + '
                
            elif (a[i] < 0 and a[i] != -1):
                
                relationStr +=  str(-a[i])  + 'd'  + ''.join(str(j) for j in face) + ' + '
      
            i += 1
        
        print(relationStr[0:-2] + ' <=  ' + str(b))

In [19]:
# delta, a simplicial complex on [n] given by its facets
# S, a subset of [n]
# a, a linear functional given as a list
# b, a constant
# switches the inequality ax \leq b of Gcut(delta) with respect to the set S
def switchIneq(delta, S, a, b):
    
    newIneq = []
    
    i = 0
    
    for face in gradedLexFaces(delta)[1:]:
        
        faceCoord = ((-1)^(Set(S).intersection(Set(face)).cardinality()))*a[i]
        
        newIneq.append(faceCoord)
        
        i += 1
        
    
    return [newIneq, b + vector(a).dot_product(vector(genCutVert(delta, S)))]

In [20]:
# delta, a simplicial complex on [n] given by its facets
# returnIneqs, False by default, if true it returns the inequaities in both string form and as pairs [a,b]
# for the inequality ax \leq b, otherwise this method returns nothing
# printIneqs, True by default, if true the inequalities are printed out otherwise nothing is printed. 
# gives the inequalities of Gcut(delta) modulo switching
# does this by starting with inequalities that do not contain the origin and computing
# all valid switchings of them. Continues this way until all inequalities are obtained
def ineqsModSwitch(delta, returnIneqs = False, printIneqs = True):
    
    ineqs = []
    
    hRep = genCutPolytope(delta).Hrep_generator()
    
    uniqueIneqs = []
    
    
    # convert ineqs to nice form and sort them so the "types" are at the top
    
    for relation in hRep:
        
        a = list(relation.A())
        b = relation.b()
        
        ineqs.append([a, b])
    
    ineqs = sorted(ineqs, key = lambda k: [-k[1], k[0]])
    
    #reduce mod switching
    
    while len(ineqs) > 0:
        
        curIneq = ineqs[0]
        
        uniqueIneqs.append(curIneq)
        
        switchings = []
        
        for S in Subsets(SimplicialComplex(delta).vertices()):
            
            switchings.append(switchIneq(delta, Set(S), curIneq[0], curIneq[1]))
        
        ineqs = [ineq for ineq in ineqs if ineq not in switchings]
        
    # print out the unique ineqs
    
    if ( printIneqs ):
    
        print(len(uniqueIneqs))

        for relation in uniqueIneqs:

            a = relation[0]
            b = relation[1]


            i = 0
            relationStr = ''

            for face in gradedLexFaces(delta)[1:]:

                if (a[i] > 0 and a[i] == 1):

                    relationStr += '-d' + ''.join(str(j) for j in face) + ' + '

                elif (a[i] > 0 and a[i] != 1):

                    relationStr += str(-a[i]) + 'd' + ''.join(str(j) for j in face) + ' + '


                elif (a[i] < 0 and a[i] == -1):

                    relationStr += 'd' + ''.join(str(j) for j in face) + ' + '

                elif (a[i] < 0 and a[i] != -1):

                    relationStr +=  str(-a[i])  + 'd'  + ''.join(str(j) for j in face) + ' + '

                i += 1

            print(relationStr[0:-2] + ' <=  ' + str(b))
    
    if ( returnIneqs ):
        
        ineqStrings = []

        for relation in uniqueIneqs:

            a = relation[0]
            b = relation[1]


            i = 0
            relationStr = ''

            for face in gradedLexFaces(delta)[1:]:

                if (a[i] > 0 and a[i] == 1):

                    relationStr += '-d' + ''.join(str(j) for j in face) + ' + '

                elif (a[i] > 0 and a[i] != 1):

                    relationStr += str(-a[i]) + 'd' + ''.join(str(j) for j in face) + ' + '


                elif (a[i] < 0 and a[i] == -1):

                    relationStr += 'd' + ''.join(str(j) for j in face) + ' + '

                elif (a[i] < 0 and a[i] != -1):

                    relationStr +=  str(-a[i])  + 'd'  + ''.join(str(j) for j in face) + ' + '

                i += 1

            ineqStrings.append(relationStr[0:-2] + ' <=  ' + str(b))
        
        return [uniqueIneqs, ineqStrings]

In [1]:
# delta, a simplicial complex on [n] given by its facets
# computes the coface description of Gcut(delta)
def vertsNotOnFacets(delta):
    
    uniqueIneqs = ineqsModSwitch(delta, True, False)
    
    eqns = uniqueIneqs[0]
    
    strings = uniqueIneqs[1]
    
    j = 0
    
    for facet in eqns:
        
        a = facet[0]
        
        b = facet[1]
        
        vertsNotOnFacet = []
        
        i = 0
        
        for col in genCutMatrix(delta).columns():
            
            if ( (-vector(a)).dot_product(vector(col)) < b ):
                
                vertsNotOnFacet.append(Subsets(SimplicialComplex(delta).vertices())[i])
                
            i += 1
        
        print(strings[j])
        print(vertsNotOnFacet)
        print()
        print()
        
        j += 1

In [22]:
# delta, a simplicial complex on [n] given as a list of facets
# returns the matrix of the inverse of the generalized covariance map
def genCovMapInv(delta):
    
    phiInv = []

    for i in range(len(gradedLexFaces(delta)[1:])):

        rowFace = gradedLexFaces(delta)[1:][i]

        curRow = []

        for j in range(len(gradedLexFaces(delta)[1:])):

            columnFace = gradedLexFaces(delta)[1:][j]

            curRow.append(phi[i][j]/(2^(rowFace.dimension() + columnFace.dimension())))

        phiInv.append(curRow) 
        

    return matrix(phiInv)

In [23]:
# delta, a simplicial complex on [n] given as a list of facets
# returns the design matrix of the hierarchical model for delta after shifting it to the origin
def shiftHierMatrix(delta):
    
    simpComp = SimplicialComplex(delta)
    
    mat = []
    
    for S in Subsets(simpComp.vertices()):
        
        col = []
        
        for F in simpComp.facets():
            
            colF = []
            
            for T in Subsets(list(F)):
                
                entryFT = 0
                
                if ( Set(S).intersection(Set(F)) == Set(T) and not Set(T).is_empty() ):
                    
                    entryFT = 1
                    
                elif ( Set(S).intersection(Set(F)) != Set(T) and Set(T).is_empty() ):
                    
                    entryFT = -1
                    
                colF.append(entryFT)
                
            col += colF
        
        mat.append(col)
        
    return Matrix(mat).transpose()

In [2]:
# delta, a simplicial complex on [n] given as a list of facets
# returns the toric ideal associated to the generalized cut polytope
def genCutIdeal(delta):
    
    varNames = 'zN, '
    
    simpComp = SimplicialComplex(delta)
    
    for S in  Subsets(simpComp.vertices())[1:]:
        
        subscript = ''
        
        for s in S:
            
            subscript += str(s)
            
            
        if ( len(S) < len(simpComp.vertices()) ):
        
            varNames += 'z' + subscript + ', '
            
        else:
            
            varNames += 'z' + subscript
            
    print(varNames)
    
    A = genCutMatrix(delta)
    
    homA = A.insert_row(0, [1 for i in range(A.ncols())])
    
    return ToricIdeal(homA, names = varNames)

In [26]:
# delta, a simplicial complex on [n] given as a list of facets
# returns the lawrence lift of delta given as a list of facets
def lawLift(delta):

    lift = []
    
    simpComp = SimplicialComplex(delta)
    
    n = simpComp.vertices()[-1]
    
    for facet in delta:
        
        lift.append(list(facet) + [n+1])
    
    lift.append(list(range(1,n+1)))
    
    return sorted(lift)

In [27]:
# delta, a simplicial complex on [n] given as a list of facets
# returns the cone over delta with p additional points added.
# the result is a simplicial complex on [n+p] 
def cone(p, delta):
    
    cone = []
    
    simpComp = SimplicialComplex(delta)
    
    n = simpComp.vertices()[-1]
    
    u = list(range(n+1, n + p + 1))
    
    for facet in delta:
        
        cone.append(facet + u)
        
    return cone

In [28]:
# delta, a simplicial complex on [n] given as a list of facets
# returns the alexander dual of delta
def alexDual(delta):
    
    simpComp = SimplicialComplex(delta).alexander_dual()
    
    facets = []
    
    for facet in simpComp.facets():
        
        facets.append(list(facet))
    
    return facets

In [29]:
# delta, a simplicial complex on [n] given as a list of facets
# computes the Gale transform of Gcut(delta)
def galeTransform(delta): 
    
    simpComp = SimplicialComplex(delta)
    
    simplex = SimplicialComplex(Subsets(simpComp.vertices()))
    
    missingFaces = sorted(Set(gradedLexFaces(simplex)).difference(Set(gradedLexFaces(delta))), key = lambda k: [len(list(k)), k])
    
    k = len(missingFaces)
    
    galeTransform = []
    
    e = identity_matrix(k)
    
    for A in Subsets(simpComp.vertices()):
        
        b = vector([0 for i in range(k)])
        
        i = 0
        
        for F in missingFaces:
            
            if ( Set(A).issubset(Set(F)) ):
                
                b = b + e.row(i)
                
            i += 1
        
        b = (-1)^(len(A))*b
        
        galeTransform.append(b)
        
    return galeTransform

In [30]:
# delta, a simplicial complex on [n] given as a list of facets
# verts, a set of vertices that form a facial set of Gcut(delta)
# returns the inequality corresponding to the facial set verts
def gCutFace(delta, verts):
    
    varsToUse = []
    
    for face in gradedLexFaces(delta)[1:]:
        
        varToUse = 'd'
        
        for el in face:
            
            varToUse += str(el)
    
        varsToUse.append(varToUse)
    
    vSet = [genCutVert(delta, v) for v in verts]
    
    mat = []
    
    for v in vSet:
        
        mat.append(v + [1])
    
    
    mat = matrix(mat)
    
    kv = mat.right_kernel().gens()[0]
    
    print(str(vector(var(varsToUse)).dot_product(kv[:-1])) + str(' = ') + str(-kv[-1]))